# CLARA + HDC MEMORY INTEGRATION (v2)

This notebook integrates Hyperdimensional Computing (HDC) memory into Clara's dual-brain architecture.

**v2 Changes:**
- Fixed memory storage: stores actual conversational content, not meta-summaries
- Lowered recall threshold (0.25 → 0.15) for better conversational follow-ups
- Added entity extraction for key conversational elements
- Stores both user queries AND Clara's responses
- Better debug output to track memory usage

**Prerequisites:**
- Trained models in Google Drive (`/Lily/models/`)
- `clara-knowledge` (Phi-3 merged)
- `mistral_warmth`, `mistral_playful`, `mistral_encouragement` adapters

**Architecture:**
```
User Query → HDC Memory Context → Semantic Router → Brain Selection → Response
                    ↑                                                    │
                    └────────────── Store Interaction ──────────────────┘
```

## Cell 1: Setup & Installation

In [ ]:
# ============================================================
# SETUP
# ============================================================

!pip install -q transformers accelerate bitsandbytes
!pip install -q peft sentence-transformers

from google.colab import drive
drive.mount('/content/drive')

import torch
import os

print("=" * 60)
print("SETUP COMPLETE")
print("=" * 60)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Cell 2: Check Existing Models (Safety Check)

In [ ]:
# ============================================================
# SAFETY CHECK - Verify existing models without overwriting
# ============================================================

MODELS_DIR = "/content/drive/MyDrive/Lily/models"
MEMORY_DIR = "/content/drive/MyDrive/Lily/memory"

print("=" * 60)
print("MODEL VERIFICATION")
print("=" * 60)

# Required models
required_models = {
    "clara-knowledge": "Phi-3 merged knowledge brain",
    "mistral_warmth": "Personality adapter (warmth)",
    "mistral_playful": "Personality adapter (playful)", 
    "mistral_encouragement": "Personality adapter (encouragement)",
}

all_present = True
model_status = {}

print("\n📁 Checking models in:", MODELS_DIR)
print("-" * 60)

for model_name, description in required_models.items():
    path = os.path.join(MODELS_DIR, model_name)
    
    # Check for key files that indicate a valid model
    if model_name == "clara-knowledge":
        # Merged model should have config.json
        check_file = os.path.join(path, "config.json")
    else:
        # LoRA adapters have adapter_config.json
        check_file = os.path.join(path, "adapter_config.json")
    
    exists = os.path.exists(check_file)
    model_status[model_name] = exists
    
    if exists:
        # Get size
        total_size = sum(
            os.path.getsize(os.path.join(path, f))
            for f in os.listdir(path)
            if os.path.isfile(os.path.join(path, f))
        ) / 1e9
        print(f"  ✅ {model_name:<25} ({total_size:.2f} GB)")
        print(f"      └─ {description}")
    else:
        print(f"  ❌ {model_name:<25} MISSING!")
        print(f"      └─ {description}")
        all_present = False

print("-" * 60)

if all_present:
    print("\n✅ All required models found!")
    print("   Models will NOT be overwritten.")
else:
    print("\n⚠️  Some models are missing!")
    print("   Please run your training notebook first.")
    print("   This notebook requires pre-trained models.")

# Check/create memory directory
print("\n📁 Memory directory:", MEMORY_DIR)
if not os.path.exists(MEMORY_DIR):
    os.makedirs(MEMORY_DIR)
    print("   Created new memory directory")
else:
    # Check for existing memory files
    memory_files = [f for f in os.listdir(MEMORY_DIR) if f.endswith('.json')]
    if memory_files:
        print(f"   Found {len(memory_files)} existing memory file(s):")
        for mf in memory_files:
            size = os.path.getsize(os.path.join(MEMORY_DIR, mf)) / 1024
            print(f"      └─ {mf} ({size:.1f} KB)")
    else:
        print("   No existing memory files (fresh start)")

## Cell 3: Load Semantic Router (Embedder)

In [ ]:
# ============================================================
# SEMANTIC ROUTER - Load embedder and domain descriptions
# ============================================================

from sentence_transformers import SentenceTransformer
import numpy as np

print("=" * 60)
print("LOADING SEMANTIC ROUTER")
print("=" * 60)

print("\n1. Loading embedding model...")
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("   ✅ all-MiniLM-L6-v2 loaded")

# Domain descriptions (refined from your notebook)
DOMAIN_DESCRIPTIONS = {
    "medical": """
        symptoms diagnosis treatment disease illness pain fever infection
        headache nauseous dizzy blood pressure heart lungs brain body
        doctor hospital medicine medication prescription surgery vaccine
        virus bacteria immune system allergies chronic acute patient health
        tired fatigue exhausted sleep insomnia rash swollen sore throat
        cough breathing chest stomach ache injury wound bleeding
    """,
    
    "coding": """
        programming code software python javascript java function method
        variable array list dictionary tuple loop error exception bug debug
        API database SQL server backend frontend algorithm data structure
        class object inheritance compile runtime syntax IndexError TypeError
        iterate parse return import library framework git repository
        crash deploy package module script terminal command line
        Flask Django React Node npm pip install developer
    """,
    
    "teaching": """
        explain how does work basics fundamentals introduction tutorial
        step by step concept theory lesson learn teach education student
        example analogy walk through ELI5 for dummies guide overview
        what is the difference between simple explanation textbook
        homework assignment class course study
    """,
    
    "quantum": """
        quantum physics qubit superposition entanglement wave function
        particle measurement collapse observer Schrodinger Heisenberg
        quantum computer quantum gate Hadamard CNOT quantum circuit
        coherence decoherence probability amplitude interference
        quantum mechanics quantum state Planck photon electron spin
        two places at once two states uncertainty principle
        parallel universes both alive and dead particle wave duality
    """,
    
    "personality": """
        feeling emotion mood happy sad angry anxious worried stressed
        excited nervous scared lonely depressed overwhelmed frustrated
        relationship friend family love support talk vent chat
        my day rough tough great amazing terrible celebrate
        broke up promotion new job interview date
        grateful appreciate thankful thanks thank you
        hey hi hello how are you good morning good night
        nobody understands me moving away miss you
        sorry to hear congratulations best friend
        take care see you later nice to meet you
        coffee tea drink offer gave give
    """,
}

print("\n2. Computing domain embeddings...")
domain_embeddings = {}
for domain, description in DOMAIN_DESCRIPTIONS.items():
    clean_desc = " ".join(description.split())
    domain_embeddings[domain] = embedder.encode(clean_desc)
    print(f"   ✅ {domain}")

print("\n✅ Semantic router ready")

## Cell 4: HDC Memory System (v2 - Fixed Recall)

In [ ]:
# ============================================================
# HDC MEMORY SYSTEM (v2 - Fixed Recall)
# ============================================================

from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Optional, Set
import time
import json
import re

print("=" * 60)
print("HDC MEMORY SYSTEM v2")
print("=" * 60)

@dataclass
class Memory:
    """Single memory unit with richer metadata"""
    text: str
    timestamp: float
    memory_type: str  # 'user_input', 'clara_response', 'preference', 'fact'
    importance: float = 0.5
    domain: str = "general"
    entities: List[str] = field(default_factory=list)  # Extracted entities
    turn_id: int = 0  # Conversation turn number


class ClaraHDCMemory:
    """
    Hyperdimensional Computing Memory System for Clara (v2)
    
    v2 Improvements:
    - Stores actual conversational content (not meta-summaries)
    - Entity extraction for key nouns/concepts
    - Lower recall threshold (0.15) for conversational follow-ups
    - Turn tracking for conversation coherence
    - Debug mode for troubleshooting
    
    HDC Operations:
    - Bind (⊗): Element-wise multiplication - creates associations
    - Bundle (+): Element-wise addition + sign - superposition
    - Similarity: Cosine similarity for retrieval
    """
    
    # Recall threshold - LOWERED from 0.25 to 0.15 for better conversational recall
    RECALL_THRESHOLD = 0.15
    
    # Memory boost threshold for routing
    MEMORY_BOOST_THRESHOLD = 0.20
    
    def __init__(self, embedder, dim: int = 10000, seed: int = 42, debug: bool = False):
        self.dim = dim
        self.embedder = embedder
        self.rng = np.random.RandomState(seed)
        self.debug = debug
        self.current_turn = 0
        
        # Random projection matrix: 384-dim → 10K-dim
        print("   Initializing projection matrix...")
        self.projection = self.rng.randn(384, dim).astype(np.float32)
        self.projection /= np.linalg.norm(self.projection, axis=1, keepdims=True)
        
        # Memory stores
        self.memories: List[Tuple[np.ndarray, Memory]] = []
        self.memory_bundle = np.zeros(dim, dtype=np.float32)
        
        # Symbol library for structured binding
        print("   Building symbol library...")
        self.symbols: Dict[str, np.ndarray] = {}
        self._init_symbols()
        
        # Entity index for fast lookup
        self.entity_index: Dict[str, List[int]] = {}  # entity -> memory indices
        
        # Personality vectors
        print("   Encoding personality vectors...")
        self.personality = self._init_personality()
        
        print(f"   ✅ HDC Memory v2 initialized (dim={dim})")
        print(f"   📊 Recall threshold: {self.RECALL_THRESHOLD}")
    
    def _init_symbols(self):
        """Create base symbol vocabulary"""
        base_symbols = [
            # Roles
            "ROLE_USER", "ROLE_CLARA", "ROLE_TOPIC", "ROLE_OUTCOME", "ROLE_ENTITY",
            # Actions  
            "ASKED", "ANSWERED", "OFFERED", "RECEIVED", "DISCUSSED",
            # Domains
            "MEDICAL", "CODING", "TEACHING", "QUANTUM", "PERSONALITY",
            # Common conversational entities
            "COFFEE", "TEA", "FOOD", "DRINK", "HELP", "THANKS",
            # Outcomes
            "HELPFUL", "CONFUSED", "SATISFIED", "FRUSTRATED",
            # Time markers
            "RECENT", "TODAY", "THIS_SESSION", "EARLIER"
        ]
        for s in base_symbols:
            self.symbols[s] = self._random_hv()
    
    def _init_personality(self) -> Dict[str, np.ndarray]:
        """Encode Clara's personality traits as hypervectors"""
        traits = {
            'warmth': 0.85,
            'curiosity': 0.75,
            'patience': 0.90,
            'encouragement': 0.80,
        }
        personality = {}
        for trait, strength in traits.items():
            base_hv = self._random_hv()
            personality[trait] = (base_hv * strength).astype(np.float32)
        
        all_traits = list(personality.values())
        personality['composite'] = np.sign(
            np.sum(all_traits, axis=0)
        ).astype(np.float32)
        
        return personality
    
    def _random_hv(self) -> np.ndarray:
        """Generate random bipolar hypervector {-1, +1}"""
        return self.rng.choice([-1, 1], size=self.dim).astype(np.float32)
    
    def _text_to_hv(self, text: str) -> np.ndarray:
        """Convert text to hypervector using existing embedder"""
        embedding = self.embedder.encode(text)
        hv = embedding @ self.projection
        return np.sign(hv).astype(np.float32)
    
    def _get_symbol(self, name: str) -> np.ndarray:
        """Get or create symbol"""
        name_upper = name.upper().replace(" ", "_")
        if name_upper not in self.symbols:
            self.symbols[name_upper] = self._random_hv()
        return self.symbols[name_upper]
    
    def _extract_entities(self, text: str) -> List[str]:
        """
        Extract key entities from text for indexing
        Simple extraction - looks for nouns and key terms
        """
        # Common conversational entities to look for
        entity_patterns = [
            r'\bcoffee\b', r'\btea\b', r'\bwater\b', r'\bdrink\b',
            r'\bfood\b', r'\blunch\b', r'\bdinner\b', r'\bbreakfast\b',
            r'\bproject\b', r'\bwork\b', r'\bmeeting\b', r'\btask\b',
            r'\bDocker\b', r'\bPython\b', r'\bcode\b', r'\bnotebook\b',
            r'\bhelp\b', r'\bthanks\b', r'\bplease\b',
        ]
        
        entities = []
        text_lower = text.lower()
        
        for pattern in entity_patterns:
            if re.search(pattern, text_lower):
                # Extract the matched word
                match = re.search(pattern, text_lower)
                if match:
                    entities.append(match.group().upper())
        
        return list(set(entities))  # Dedupe
    
    # === HDC Operations ===
    
    def bind(self, hv1: np.ndarray, hv2: np.ndarray) -> np.ndarray:
        """Bind two hypervectors (⊗) - creates association"""
        return hv1 * hv2
    
    def bundle(self, hvs: List[np.ndarray]) -> np.ndarray:
        """Bundle hypervectors (+) - superposition"""
        if not hvs:
            return np.zeros(self.dim, dtype=np.float32)
        return np.sign(np.sum(hvs, axis=0)).astype(np.float32)
    
    def similarity(self, hv1: np.ndarray, hv2: np.ndarray) -> float:
        """Cosine similarity between hypervectors"""
        n1, n2 = np.linalg.norm(hv1), np.linalg.norm(hv2)
        if n1 == 0 or n2 == 0:
            return 0.0
        return float(np.dot(hv1, hv2) / (n1 * n2))
    
    # === Memory Operations ===
    
    def store(self, text: str, memory_type: str = "user_input",
              importance: float = 0.5, domain: str = "general",
              increment_turn: bool = False, **bindings) -> None:
        """
        Store a memory with entity extraction and optional bindings
        
        Args:
            text: The actual conversational content (NOT a summary!)
            memory_type: 'user_input', 'clara_response', 'preference', 'fact'
            importance: 0.0-1.0 importance score
            domain: 'medical', 'coding', 'teaching', 'quantum', 'personality'
            increment_turn: Whether this starts a new conversation turn
            **bindings: Structured role-filler pairs
        """
        if increment_turn:
            self.current_turn += 1
        
        # Extract entities from text
        entities = self._extract_entities(text)
        
        # Create semantic hypervector from ACTUAL text
        text_hv = self._text_to_hv(text)
        
        # Add domain binding
        domain_hv = self._get_symbol(domain.upper())
        text_hv = self.bind(text_hv, domain_hv)
        
        # Add entity bindings
        if entities:
            entity_hvs = []
            for entity in entities:
                entity_hv = self._get_symbol(entity)
                role_hv = self._get_symbol("ROLE_ENTITY")
                entity_hvs.append(self.bind(role_hv, entity_hv))
            if entity_hvs:
                entity_bundle = self.bundle(entity_hvs)
                text_hv = self.bundle([text_hv, entity_bundle])
        
        # Add structural bindings if provided
        if bindings:
            bound_parts = []
            for role, filler in bindings.items():
                role_hv = self._get_symbol(f"ROLE_{role.upper()}")
                filler_hv = self._get_symbol(str(filler).upper())
                bound_parts.append(self.bind(role_hv, filler_hv))
            if bound_parts:
                structure_hv = self.bundle(bound_parts)
                text_hv = self.bundle([text_hv, structure_hv])
        
        memory = Memory(
            text=text,
            timestamp=time.time(),
            memory_type=memory_type,
            importance=importance,
            domain=domain,
            entities=entities,
            turn_id=self.current_turn
        )
        
        memory_idx = len(self.memories)
        self.memories.append((text_hv, memory))
        
        # Update entity index
        for entity in entities:
            if entity not in self.entity_index:
                self.entity_index[entity] = []
            self.entity_index[entity].append(memory_idx)
        
        # Update bundled representation
        self.memory_bundle = self.bundle([self.memory_bundle, text_hv])
        
        if self.debug:
            print(f"      [MEM] Stored: '{text[:50]}...'")
            print(f"            Entities: {entities}")
            print(f"            Turn: {self.current_turn}, Importance: {importance:.2f}")
    
    def recall(self, query: str, top_k: int = 5,
               domain_filter: Optional[str] = None,
               include_entities: bool = True) -> List[Tuple[Memory, float]]:
        """
        Retrieve memories similar to query
        
        v2: Also boosts memories that share entities with the query
        """
        if not self.memories:
            return []
        
        query_hv = self._text_to_hv(query)
        query_entities = self._extract_entities(query) if include_entities else []
        
        if self.debug:
            print(f"      [RECALL] Query: '{query[:40]}...'")
            print(f"               Entities: {query_entities}")
        
        results = []
        for idx, (hv, memory) in enumerate(self.memories):
            # Apply domain filter if specified
            if domain_filter and memory.domain != domain_filter:
                continue
            
            # Base similarity
            sim = self.similarity(query_hv, hv)
            
            # Entity overlap boost
            entity_boost = 0.0
            if query_entities and memory.entities:
                overlap = set(query_entities) & set(memory.entities)
                if overlap:
                    entity_boost = 0.15 * len(overlap)  # Boost per shared entity
                    if self.debug:
                        print(f"               Entity match: {overlap} (+{entity_boost:.2f})")
            
            # Recency boost (stronger for recent turns)
            turn_diff = self.current_turn - memory.turn_id
            recency = 1.0 / (1.0 + turn_diff * 0.1)  # Slower decay
            
            # Time-based recency (for cross-session)
            age_hours = (time.time() - memory.timestamp) / 3600
            time_recency = 1.0 / (1.0 + age_hours / 24)
            
            # Combined score
            weighted = (
                sim + entity_boost
            ) * (0.6 + 0.2 * memory.importance) * (0.7 + 0.15 * recency + 0.15 * time_recency)
            
            results.append((memory, weighted))
            
            if self.debug and sim > 0.1:
                print(f"               [{idx}] sim={sim:.3f} final={weighted:.3f} '{memory.text[:30]}...'")
        
        results.sort(key=lambda x: x[1], reverse=True)
        return results[:top_k]
    
    def recall_by_entity(self, entity: str) -> List[Memory]:
        """Fast lookup by entity name"""
        entity_upper = entity.upper()
        if entity_upper not in self.entity_index:
            return []
        
        indices = self.entity_index[entity_upper]
        return [self.memories[i][1] for i in indices]
    
    def get_context_for_routing(self, query: str) -> np.ndarray:
        """
        Get memory-augmented context for routing decisions
        """
        query_hv = self._text_to_hv(query)
        
        relevant = self.recall(query, top_k=3)
        if relevant:
            memory_hvs = [self._text_to_hv(m.text) for m, _ in relevant]
            memory_context = self.bundle(memory_hvs)
        else:
            memory_context = np.zeros(self.dim, dtype=np.float32)
        
        routing_hv = self.bundle([
            query_hv,
            memory_context * 0.3,
            self.personality['composite'] * 0.2
        ])
        
        return routing_hv
    
    def get_context_string(self, query: str, max_memories: int = 3) -> str:
        """
        Get memory context as string for prompt injection
        
        v2: Lower threshold, better formatting
        """
        relevant = self.recall(query, top_k=max_memories)
        if not relevant:
            return ""
        
        # Use class threshold constant
        good_memories = [(m, s) for m, s in relevant if s > self.RECALL_THRESHOLD]
        
        if self.debug:
            print(f"      [CONTEXT] {len(relevant)} retrieved, {len(good_memories)} above threshold ({self.RECALL_THRESHOLD})")
        
        if not good_memories:
            return ""
        
        context_parts = ["[Conversation context:"]
        for mem, score in good_memories:
            # Format based on memory type
            if mem.memory_type == "user_input":
                context_parts.append(f"- User said: {mem.text[:100]}")
            elif mem.memory_type == "clara_response":
                context_parts.append(f"- Clara said: {mem.text[:100]}")
            else:
                context_parts.append(f"- {mem.text[:100]}")
        context_parts.append("]")
        
        return "\n".join(context_parts)
    
    def get_recent_context(self, n_turns: int = 2) -> str:
        """
        Get context from most recent N conversation turns
        (Fallback for when semantic search doesn't find matches)
        """
        if not self.memories:
            return ""
        
        min_turn = max(0, self.current_turn - n_turns)
        recent = [
            m for _, m in self.memories 
            if m.turn_id >= min_turn
        ]
        
        if not recent:
            return ""
        
        context_parts = ["[Recent conversation:"]
        for mem in recent[-6:]:  # Last 6 memories max
            if mem.memory_type == "user_input":
                context_parts.append(f"- User: {mem.text[:80]}")
            elif mem.memory_type == "clara_response":
                context_parts.append(f"- Clara: {mem.text[:80]}")
        context_parts.append("]")
        
        return "\n".join(context_parts)
    
    def stats(self) -> Dict:
        """Get memory statistics"""
        domain_counts = {}
        type_counts = {}
        for _, m in self.memories:
            domain_counts[m.domain] = domain_counts.get(m.domain, 0) + 1
            type_counts[m.memory_type] = type_counts.get(m.memory_type, 0) + 1
        
        return {
            "total_memories": len(self.memories),
            "by_domain": domain_counts,
            "by_type": type_counts,
            "symbols": len(self.symbols),
            "entities_tracked": len(self.entity_index),
            "current_turn": self.current_turn,
            "size_kb": self.size_bytes() / 1024
        }
    
    def size_bytes(self) -> int:
        """Memory footprint (excluding embedder)"""
        bundle_size = self.memory_bundle.nbytes
        memories_size = sum(hv.nbytes for hv, _ in self.memories)
        symbols_size = sum(hv.nbytes for hv in self.symbols.values())
        personality_size = sum(hv.nbytes for hv in self.personality.values())
        projection_size = self.projection.nbytes
        return bundle_size + memories_size + symbols_size + personality_size + projection_size
    
    # === Persistence ===
    
    def save(self, path: str):
        """Save memory state to file"""
        data = {
            'version': '2.0',
            'dim': self.dim,
            'current_turn': self.current_turn,
            'recall_threshold': self.RECALL_THRESHOLD,
            'memories': [
                {
                    'hv': hv.tolist(),
                    'text': m.text,
                    'timestamp': m.timestamp,
                    'memory_type': m.memory_type,
                    'importance': m.importance,
                    'domain': m.domain,
                    'entities': m.entities,
                    'turn_id': m.turn_id
                }
                for hv, m in self.memories
            ],
            'symbols': {k: v.tolist() for k, v in self.symbols.items()},
            'entity_index': self.entity_index,
            'memory_bundle': self.memory_bundle.tolist(),
            'projection': self.projection.tolist()
        }
        
        with open(path, 'w') as f:
            json.dump(data, f)
        
        print(f"✅ Saved {len(self.memories)} memories to {path}")
        print(f"   Size: {os.path.getsize(path) / 1024:.1f} KB")
    
    def load(self, path: str) -> bool:
        """Load memory state from file"""
        if not os.path.exists(path):
            print(f"⚠️ No memory file found at {path}")
            return False
        
        with open(path) as f:
            data = json.load(f)
        
        # Check version
        version = data.get('version', '1.0')
        print(f"   Loading memory file version {version}")
        
        if data.get('dim') != self.dim:
            print(f"⚠️ Dimension mismatch: file has {data.get('dim')}, expected {self.dim}")
            return False
        
        # Load memories
        self.memories = []
        for m in data['memories']:
            memory = Memory(
                text=m['text'],
                timestamp=m['timestamp'],
                memory_type=m.get('memory_type', 'interaction'),
                importance=m['importance'],
                domain=m.get('domain', 'general'),
                entities=m.get('entities', []),
                turn_id=m.get('turn_id', 0)
            )
            hv = np.array(m['hv'], dtype=np.float32)
            self.memories.append((hv, memory))
        
        # Load other state
        self.symbols = {k: np.array(v, dtype=np.float32) for k, v in data['symbols'].items()}
        self.entity_index = data.get('entity_index', {})
        self.memory_bundle = np.array(data['memory_bundle'], dtype=np.float32)
        self.current_turn = data.get('current_turn', 0)
        
        if 'projection' in data:
            self.projection = np.array(data['projection'], dtype=np.float32)
        
        print(f"✅ Loaded {len(self.memories)} memories from {path}")
        return True


# ============================================================
# Initialize HDC Memory
# ============================================================
print("\nInitializing Clara's HDC Memory v2...")

# Set debug=True to see memory operations
DEBUG_MEMORY = True

clara_memory = ClaraHDCMemory(embedder=embedder, dim=10000, debug=DEBUG_MEMORY)

# Try to load existing memory
MEMORY_FILE = os.path.join(MEMORY_DIR, "clara_memory_v2.json")
if os.path.exists(MEMORY_FILE):
    print(f"\nFound existing memory file, loading...")
    clara_memory.load(MEMORY_FILE)
else:
    print(f"\nNo existing memory file - starting fresh")

print(f"\n📊 Memory Stats:")
stats = clara_memory.stats()
print(f"   Total memories: {stats['total_memories']}")
print(f"   Current turn: {stats['current_turn']}")
print(f"   Entities tracked: {stats['entities_tracked']}")
print(f"   Symbols: {stats['symbols']}")
print(f"   Size: {stats['size_kb']:.1f} KB")

## Cell 5: Smart Router with Memory (v2)

In [ ]:
# ============================================================
# SMART ROUTER v2 (with improved memory integration)
# ============================================================

print("=" * 60)
print("SMART ROUTER v2")
print("=" * 60)

def smart_route(query: str, use_memory: bool = True, 
                threshold: float = 0.20) -> tuple:
    """
    Route query using semantic similarity + memory context
    
    v2: Lower memory boost threshold (0.30 → 0.20)
    """
    query_embedding = embedder.encode(query)
    
    similarities = {}
    for domain, domain_emb in domain_embeddings.items():
        similarity = np.dot(query_embedding, domain_emb) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(domain_emb)
        )
        similarities[domain] = similarity
    
    # Memory-based boosting with LOWER threshold
    memory_context_used = False
    if use_memory and len(clara_memory.memories) > 0:
        relevant = clara_memory.recall(query, top_k=2)
        if relevant:
            for mem, score in relevant:
                # v2: Lowered from 0.30 to 0.20
                if score > clara_memory.MEMORY_BOOST_THRESHOLD and mem.domain in similarities:
                    similarities[mem.domain] += 0.05
                    memory_context_used = True
    
    sorted_domains = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    best_domain, best_conf = sorted_domains[0]
    second_domain, second_conf = sorted_domains[1]
    
    # If close, prefer personality (warmth)
    if best_conf - second_conf < 0.05:
        if second_domain == "personality":
            best_domain = "personality"
            best_conf = second_conf
    
    # Low confidence fallback
    if best_conf < threshold and best_domain != "personality":
        if similarities["personality"] > 0.15:
            best_domain = "personality"
            best_conf = similarities["personality"]
    
    if best_domain == "personality":
        brain = "personality"
        domain = "warmth"
    else:
        brain = "knowledge"
        domain = best_domain
    
    return brain, domain, best_conf, memory_context_used


def clean_response(response: str) -> str:
    """Clean up response artifacts"""
    stop_markers = ["### Instruction:", "Instruction:", "\n\n\n", "User:", "[Conversation context:"]
    for marker in stop_markers:
        if marker in response:
            response = response.split(marker)[0].strip()
    return response.strip()


print("✅ Router v2 functions defined")

# Quick test
print("\n📋 Router test:")
test_queries = [
    "How do I read a CSV in Python?",
    "I'm feeling stressed about work",
    "What is quantum entanglement?"
]
for q in test_queries:
    brain, domain, conf, mem_used = smart_route(q)
    mem_icon = "🧠" if mem_used else "  "
    print(f"   {mem_icon} {q[:40]:40} → {brain}/{domain} ({conf:.2f})")

## Cell 6: Load Clara's Brains

In [ ]:
# ============================================================
# LOAD CLARA'S DUAL-BRAIN SYSTEM
# ============================================================

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

print("=" * 60)
print("LOADING CLARA'S BRAINS")
print("=" * 60)

# ============================================================
# KNOWLEDGE BRAIN (Phi-3 merged)
# ============================================================
print("\n1. Loading Knowledge Brain (Phi-3)...")

knowledge_path = os.path.join(MODELS_DIR, "clara-knowledge")

if not os.path.exists(knowledge_path):
    raise FileNotFoundError(f"Knowledge model not found: {knowledge_path}")

knowledge_tokenizer = AutoTokenizer.from_pretrained(
    knowledge_path,
    trust_remote_code=True
)

knowledge_model = AutoModelForCausalLM.from_pretrained(
    knowledge_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
knowledge_model.eval()
print("   ✅ Knowledge brain loaded")

# ============================================================
# PERSONALITY BRAIN (Mistral + LoRA adapters)
# ============================================================
print("\n2. Loading Personality Brain (Mistral + adapters)...")

personality_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3"
)

personality_base = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

adapters_to_load = [
    ("warmth", "mistral_warmth"),
    ("playful", "mistral_playful"),
    ("encouragement", "mistral_encouragement"),
]

first_name, first_path = adapters_to_load[0]
personality_model = PeftModel.from_pretrained(
    personality_base,
    os.path.join(MODELS_DIR, first_path),
    adapter_name=first_name
)

for adapter_name, adapter_path in adapters_to_load[1:]:
    full_path = os.path.join(MODELS_DIR, adapter_path)
    if os.path.exists(full_path):
        personality_model.load_adapter(full_path, adapter_name=adapter_name)
        print(f"   ✅ Loaded adapter: {adapter_name}")
    else:
        print(f"   ⚠️ Adapter not found: {adapter_path}")

personality_model.set_adapter("warmth")
personality_model.eval()
print("   ✅ Personality brain loaded")

print("\n" + "=" * 60)
print("🧠 CLARA'S BRAINS ARE READY!")
print("=" * 60)

## Cell 7: Clara Main Interface (v2 - Fixed Memory)

In [ ]:
# ============================================================
# CLARA - Main Interface v2 (Fixed Memory Storage & Recall)
# ============================================================

print("=" * 60)
print("CLARA - MAIN INTERFACE v2")
print("=" * 60)

def clara(query: str, verbose: bool = True, 
          use_memory: bool = True,
          store_interaction: bool = True) -> str:
    """
    Clara's main interface - v2 with fixed memory!
    
    v2 Changes:
    - Stores ACTUAL query text (not meta-summary)
    - Stores Clara's response too
    - Uses recent context as fallback
    - Better memory debug output
    """
    
    # 1. Route the query
    brain, domain, conf, mem_used = smart_route(query, use_memory=use_memory)
    
    if verbose:
        mem_icon = "🧠" if mem_used else "  "
        print(f"   {mem_icon} Routing: {brain}/{domain} (conf: {conf:.2f})")
    
    # 2. Get memory context for prompt
    memory_context = ""
    if use_memory:
        # Try semantic recall first
        memory_context = clara_memory.get_context_string(query)
        
        # If no semantic match, use recent conversation context
        if not memory_context and clara_memory.current_turn > 0:
            memory_context = clara_memory.get_recent_context(n_turns=2)
            if memory_context and verbose:
                print(f"   📚 Using recent context (no semantic match)")
        elif memory_context and verbose:
            print(f"   📚 Memory context included (semantic match)")
    
    # 3. Generate response
    if brain == "knowledge":
        if memory_context:
            prompt = f"### Instruction:\n{memory_context}\n\nUser question: {query}\n\n### Response:\n"
        else:
            prompt = f"### Instruction:\n{query}\n\n### Response:\n"
        
        inputs = knowledge_tokenizer(prompt, return_tensors="pt").to(knowledge_model.device)
        
        with torch.no_grad():
            outputs = knowledge_model.generate(
                **inputs,
                max_new_tokens=250,
                temperature=0.7,
                do_sample=True,
                pad_token_id=knowledge_tokenizer.eos_token_id,
                use_cache=False
            )
        
        response = knowledge_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()
    
    else:
        personality_model.set_adapter(domain)
        
        if memory_context:
            prompt = f"### Instruction:\n{memory_context}\n\nUser message: {query}\n\n### Response:\n"
        else:
            prompt = f"### Instruction:\n{query}\n\n### Response:\n"
        
        inputs = personality_tokenizer(prompt, return_tensors="pt").to(personality_model.device)
        
        with torch.no_grad():
            outputs = personality_model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=personality_tokenizer.eos_token_id
            )
        
        response = personality_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()
    
    response = clean_response(response)
    
    # 4. Store interaction in memory - v2: ACTUAL CONTENT!
    if store_interaction:
        effective_domain = domain if brain == "knowledge" else "personality"
        
        # Store user's actual query (increment turn)
        clara_memory.store(
            text=query,  # ACTUAL QUERY, not summary!
            memory_type="user_input",
            importance=0.5 + (conf * 0.3),
            domain=effective_domain,
            increment_turn=True
        )
        
        # Store Clara's response (truncated)
        clara_memory.store(
            text=response[:150],  # Store actual response
            memory_type="clara_response",
            importance=0.3,
            domain=effective_domain
        )
    
    return response


def clara_remember(text: str, importance: float = 0.7, 
                   domain: str = "general") -> None:
    """Explicitly store something in Clara's memory"""
    clara_memory.store(
        text=text,
        memory_type="preference",
        importance=importance,
        domain=domain
    )
    print(f"✅ Stored: {text[:50]}...")


def clara_recall(query: str, top_k: int = 5) -> None:
    """Query Clara's memory directly"""
    results = clara_memory.recall(query, top_k=top_k)
    
    print(f"\n🔍 Memory search: '{query}'")
    print("-" * 60)
    
    if not results:
        print("   No memories found")
        return
    
    for mem, score in results:
        entities_str = f" [{', '.join(mem.entities)}]" if mem.entities else ""
        print(f"   [{score:.3f}] [{mem.memory_type:14}] [{mem.domain:10}]{entities_str}")
        print(f"           '{mem.text[:60]}...'")


def clara_stats() -> None:
    """Show Clara's memory statistics"""
    stats = clara_memory.stats()
    
    print("\n📊 Clara's Memory Stats")
    print("-" * 60)
    print(f"   Total memories: {stats['total_memories']}")
    print(f"   Current turn: {stats['current_turn']}")
    print(f"   Entities tracked: {stats['entities_tracked']}")
    print(f"   Symbols: {stats['symbols']}")
    print(f"   Size: {stats['size_kb']:.1f} KB")
    
    if stats['by_type']:
        print("\n   By type:")
        for mtype, count in sorted(stats['by_type'].items()):
            print(f"      {mtype}: {count}")
    
    if stats['by_domain']:
        print("\n   By domain:")
        for domain, count in sorted(stats['by_domain'].items()):
            print(f"      {domain}: {count}")


def clara_entities() -> None:
    """Show tracked entities"""
    print("\n🏷️ Tracked Entities")
    print("-" * 60)
    for entity, indices in sorted(clara_memory.entity_index.items()):
        print(f"   {entity}: {len(indices)} mention(s)")


def clara_save() -> None:
    """Save Clara's memory to disk"""
    clara_memory.save(MEMORY_FILE)


def clara_debug(on: bool = True) -> None:
    """Toggle debug mode"""
    clara_memory.debug = on
    print(f"Debug mode: {'ON' if on else 'OFF'}")


print("\n✅ Clara interface v2 ready!")
print("\nAvailable functions:")
print("   clara(query)           - Talk to Clara")
print("   clara_remember(text)   - Store something in memory")
print("   clara_recall(query)    - Search Clara's memory")
print("   clara_stats()          - Show memory statistics")
print("   clara_entities()       - Show tracked entities")
print("   clara_save()           - Save memory to disk")
print("   clara_debug(on/off)    - Toggle debug output")

## Cell 8: Test Memory Recall (Coffee Test!)

In [ ]:
# ============================================================
# TEST MEMORY RECALL - The Coffee Test!
# ============================================================

print("=" * 60)
print("TESTING CLARA's MEMORY - The Coffee Test")
print("=" * 60)

# Make sure debug is on for this test
clara_debug(True)

# Clear any existing memories for clean test
clara_memory.memories = []
clara_memory.memory_bundle = np.zeros(clara_memory.dim, dtype=np.float32)
clara_memory.entity_index = {}
clara_memory.current_turn = 0

print("\n" + "-" * 60)
print("Turn 1: Offering coffee")
print("-" * 60)
print("\n👤 User: Hi! Would you like some coffee?")
response = clara("Hi! Would you like some coffee?")
print(f"\n🤖 Clara: {response}")

print("\n" + "-" * 60)
print("Turn 2: Different topic")
print("-" * 60)
print("\n👤 User: What's your favorite thing about helping people?")
response = clara("What's your favorite thing about helping people?")
print(f"\n🤖 Clara: {response}")

print("\n" + "-" * 60)
print("Turn 3: THE COFFEE RECALL TEST")
print("-" * 60)
print("\n👤 User: How's the coffee?")
response = clara("How's the coffee?")
print(f"\n🤖 Clara: {response}")

print("\n" + "-" * 60)
print("Memory Analysis")
print("-" * 60)

clara_stats()
clara_entities()

print("\n" + "-" * 60)
print("Direct memory search for 'coffee'")
print("-" * 60)
clara_recall("coffee")

## Cell 9: Interactive Session

In [ ]:
# ============================================================
# INTERACTIVE SESSION
# ============================================================

print("=" * 60)
print("INTERACTIVE CLARA SESSION")
print("=" * 60)
print("\nType your message to Clara.")
print("Special commands:")
print("   /memory     - Show memory stats")
print("   /recall X   - Search memory for X")
print("   /entities   - Show tracked entities")
print("   /debug      - Toggle debug mode")
print("   /save       - Save memory to disk")
print("   /clear      - Clear all memories")
print("   /quit       - End session")
print("-" * 60)

while True:
    try:
        user_input = input("\n👤 You: ").strip()
        
        if not user_input:
            continue
        
        # Handle special commands
        if user_input.lower() == "/quit":
            print("\nEnding session. Don't forget to save memory!")
            break
        
        elif user_input.lower() == "/memory":
            clara_stats()
            continue
        
        elif user_input.lower().startswith("/recall "):
            query = user_input[8:].strip()
            clara_recall(query)
            continue
        
        elif user_input.lower() == "/entities":
            clara_entities()
            continue
        
        elif user_input.lower() == "/debug":
            clara_debug(not clara_memory.debug)
            continue
        
        elif user_input.lower() == "/save":
            clara_save()
            continue
        
        elif user_input.lower() == "/clear":
            clara_memory.memories = []
            clara_memory.memory_bundle = np.zeros(clara_memory.dim, dtype=np.float32)
            clara_memory.entity_index = {}
            clara_memory.current_turn = 0
            print("Memory cleared!")
            continue
        
        # Regular conversation
        response = clara(user_input)
        print(f"\n🤖 Clara: {response}")
        
    except KeyboardInterrupt:
        print("\n\nSession interrupted.")
        break

## Cell 10: Save Memory & Cleanup

In [ ]:
# ============================================================
# SAVE MEMORY & CLEANUP
# ============================================================

print("=" * 60)
print("SAVING & CLEANUP")
print("=" * 60)

# Final stats
clara_stats()
clara_entities()

# Save memory
print("\n" + "-" * 60)
print("Saving memory...")
clara_save()

# Verify save
if os.path.exists(MEMORY_FILE):
    size = os.path.getsize(MEMORY_FILE) / 1024
    print(f"\n✅ Memory saved successfully!")
    print(f"   File: {MEMORY_FILE}")
    print(f"   Size: {size:.1f} KB")
else:
    print("\n❌ Failed to save memory")

print("\n" + "=" * 60)
print("SESSION COMPLETE")
print("=" * 60)
print("\nNext time you run this notebook, Clara will remember!")